In [ ]:
from grid.gcp import *

In [ ]:
r = create('boardlaw-1', 'boardlaw-refine-1')

In [ ]:
conn = connection('boardlaw-1')

In [ ]:
r = api().instances().list(project=PROJECT, zone='us-west1-a').execute()

In [ ]:
info = r['items'][0]

In [ ]:
name = 'boardlaw-1'

In [ ]:
conn = connection(name)
container = conn.run('docker ps -aqf "ancestor=andyljones/boardlaw"', hide=True).stdout.strip()

In [ ]:
conn.run(f'docker exec {container} ps -elf')

In [ ]:
info = instances()[name]
external_ip = info['networkInterfaces'][0]['accessConfigs'][0]['natIP']

In [ ]:
external_ip